In [1]:
from collections import deque
import heapq
from plotly import graph_objects as go
import ipywidgets as widgets
import math
import base64
import os
import csv

In [2]:
class WeightedGraph(object):
    def __init__(self, cords={}, graph={}):
        self.graph = graph
        self.result = []
        self.cords = cords

    def add_vertex(self, vertex):
        if vertex not in self.graph:
            self.graph[vertex] = {}

    def vertices(self):
        return list(self.graph.keys())

    def add_edge(self, edge, add_reversed=True):
        vertex1, vertex2, weight = edge
        self.graph[vertex1][vertex2] = weight
        if add_reversed:
            self.graph[vertex2][vertex1] = weight

    def edges(self, visual=False):
        edges = []
        for vertex in self.graph:
            for neighbour, weight in self.graph[vertex].items():
                if visual:
                    edges.append((vertex, neighbour))
                else:
                    edges.append((vertex, neighbour, weight))
        return edges

    def neighbours(self, vertex):
        return list(self.graph[vertex].items())
    
    def find_city(self, city):
        for node in self.vertices():
            if city.lower() in node.lower():
                return node

    def euclidean_distance(self, vertex1, vertex2):
        vertex1 = int(((vertex1.split('-')[0])).strip())
        vertex2 = int(((vertex2.split('-')[0])).strip())
        return math.sqrt((self.cords[vertex1][0] - self.cords[vertex2][0])**2 + (self.cords[vertex1][1] - self.cords[vertex2][1])**2)

In [3]:
minx = 7458213.82
miny = 4523483.66
cords = {}
with open('Resources/Data/roads/coordinates.csv', mode='r', encoding="utf8") as roads_file:
    reader = csv.DictReader(roads_file)
    for row in reader:
        cords[int(row["Јазол"])] = (float(row["X"]), float(row["Y"]))
city = WeightedGraph(cords)
with open('Resources/Data/roads/roads_info.csv', mode='r', encoding="utf8") as roads_file:
    reader = csv.DictReader(roads_file)
    for row in reader:
        with open('Resources/Data/roads/'+row["Кратенка"]+'.csv', mode='r',encoding="utf8") as file:
            towns = csv.DictReader(file)
            for town in towns:
                pocetok = (town['ЈАЗОЛ НА ПОЧЕТОКОТ']).strip()
                kraj = (town['ЈАЗОЛ НА КРАЈОТ']).strip()
                city.add_vertex(pocetok)
                city.add_vertex(kraj)
                city.add_edge((pocetok, kraj, int(town['ДОЛЖИНА'])))

In [4]:
def uniform_cost_search(starting_vertex, goal_vertex):
    expanded = set()
    queue = [(0, [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight, vertex_list = heapq.heappop(queue)
        vertex_to_expand = vertex_list[-1]
       
        global weight_end
        weight_end = current_path_weight
        
        if vertex_to_expand in expanded:
            continue

        frontier = list(set([q[-1][-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand
        
        for neighbour, new_weight in city.neighbours(vertex_to_expand):
            if neighbour not in expanded:
                heapq.heappush(queue, (weight + new_weight, vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)
    yield [], [], goal_vertex

In [5]:
def a_star_search(starting_vertex, goal_vertex, alpha):
    expanded = set()
    queue = [((0, 0), [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight_tupple, vertex_list = heapq.heappop(queue)
        current_a_star_weight, current_path_weight = weight_tupple
        vertex_to_expand = vertex_list[-1]

        global weight_end
        weight_end = current_path_weight

        if vertex_to_expand in expanded:
            continue

        frontier = list(set([q[-1][-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand

        for neighbour, new_weight in city.neighbours(vertex_to_expand):
            if neighbour not in expanded:
                heuristic = city.euclidean_distance(neighbour, goal_vertex)
                path_weight = current_path_weight + new_weight
                a_star_weight = path_weight + alpha * heuristic
                heapq.heappush(queue, ((a_star_weight, path_weight), vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)
    yield [], [], goal_vertex

In [6]:
def count_score(end=False):
    global count
    text.value = "<font size=4> Score: " + str(count)
    if end:
        text.value = "<font size=4> Final Score: " + str(count + weight_end )
    count += 1

In [7]:
def update_fig(x):
    if button.disabled is True:
        return
    else:
        frontier, current_path, vertex_to_expand = next(simulation)
        count_score()
        if vertex_to_expand == GOAL:
            button.disabled = True
            play.disabled = True
            play.value = play.max
            count_score(True)
            print(current_path)

    color = list(fig.data[0].marker.color)
    for i,data in enumerate(fig.data[0].text):
        if data in frontier:
            color[i] = "#ff7f0e"
        elif  data in current_path:
            color[i] = "#2ca02c"
        elif  data == GOAL or data == START:
            color[i] = "#d62728"    
        else:
            color[i] = "#1f77b4"
    with fig.batch_update():
        fig.data[0].marker.color = color  
           
            

def create_button():
    button = widgets.Button(description='Чекор', disabled = True)
    button.on_click(update_fig)
    return button

def create_score():
    score = widgets.HTML(value='<font size=4> Izberi ja destinacijata')
    return score

def create_play():
    play = widgets.Play(value=0, max=3000, interval=10, show_repeat=False, disabled = True)
    play.observe(update_fig, 'value')
    return play


def create_fig():

    fig = go.FigureWidget()
    scale_factor = 222
    img_width = 1020
    img_height = 800
    imagem_tunel = base64.b64encode(open("Resources/Photos/map.png", 'rb').read())
    
    x, y, names, color = [],[],[],[]
    for i in range(1,838):
        name = [temp for temp in city.vertices() if i == int(temp.split('-')[0])]
        names.append(name[0] if name else "NaN")
        x.append(city.cords[i][0] - minx + 7500 if i in city.cords else 0)
        y.append(city.cords[i][1] - miny + 4500 if i in city.cords else 0)
        color.append("#1f77b4")
        
    fig.add_scatter(
        x=x,
        y=y,
        marker_size=10,
        marker_color = color,
        mode='markers',
        marker_symbol = "circle",
        hoverinfo='text',
        text = names
    )   
    fig.add_layout_image(
        dict(
            x=0,
            sizex=img_width * scale_factor,
            y=img_height * scale_factor,
            sizey=img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=1.0,
            layer="below",
            sizing="stretch",
            source='data:image/png;base64,{}'.format(imagem_tunel.decode()))
    )
    fig.update_xaxes(
        visible=False,
        range=[0, img_width * scale_factor]
    )

    fig.update_yaxes(
        visible=False,
        range=[0, img_height * scale_factor],
        scaleanchor="x"
    )
    fig.update_layout(
        width=img_width,
        height=img_height,
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
        showlegend=False
    )
    
    scatter = fig.data[0]  
    def update_point(trace, points, selector):
        global br,START,GOAL
        br += 1
        if br <= 2:
            c = list(scatter.marker.color)
            name = list(scatter.text)
            for i in points.point_inds:
                c[i] = '#d62728'
                if br == 1:
                    START = name[i]
                elif br == 2:
                    GOAL = name[i]
                    global simulation
                    simulation = a_star_search(START, GOAL, 0)
                    button.disabled = False
                    play.disabled = False
                with fig.batch_update():
                    scatter.marker.color = c
                               
    scatter.on_click(update_point)
            
    return fig

In [9]:
START,GOAL = '',''
simulation = 0
if simulation:
    del simulation
count = 0
br=0
play = create_play()
button = create_button()
text = create_score()
fig = create_fig()
widgets.VBox([widgets.HBox([play, button, text]), fig])